Pra prever a nota do **IMDB** (`IMDB_Rating`) tratei o problema como uma tarefa de **regressão**, já q a variável alvo é continua (vai de 0 até 10).  

As variáveis q usei foram:  
- **Numéricas**: `Runtime` (convertido p/ minutos), `Meta_score`, `No_of_Votes`, `Gross` (ajustado p/ número).  
- **Categóricas**: `Genre`, `Certificate`, `Director` e os atores principais (`Star1`–`Star4`).  
  Nessas apliquei `OneHot` ou contagem de freq p/ transformar em número.  
- **Transformações extras**:  
  - Normalizei escalas numéricas ($MinMaxScaler$), pq atributos como $No\_of\_Votes$ e $Gross$ tem magnitudes mt diferentes.  
  - No `Genre`, como pode ter + de um valor, fiz one-hot multirótulo (cada gênero vira 0/1 separado).

O modelo q testei foi o **Random Forest Regressor**.  
- **Prós**: robusto a outliers, pega relações não lineares, n precisa assumir distribuição das variáveis.  
- **Contras**: interpretabilidade baixa e custo comp. maior q modelos lineares.  

Como métrica, escolhi o **RMSE** ($\sqrt{MSE}$) pq dá o erro médio direto na escala do IMDB.  
Além disso tb usei o $R^2$ como complemento p/ ver a proporção da variância explicada.


In [6]:
import pandas as pd


path = '..\\data\\processed\\final_df.csv'

df = pd.read_csv(path)

In [7]:
import pandas as pd
import numpy as np
import joblib
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import cross_val_score, KFold


class FeatureCreator(BaseEstimator, TransformerMixin):
    def __init__(self, top_n_genres=15):
        self.top_n_genres = top_n_genres
        self.top_genres_ = []
        self.max_year_ = 0

    def fit(self, X, y=None):
        genres = X['Genre'].str.split(', ').explode()
        self.top_genres_ = genres.value_counts().nlargest(self.top_n_genres).index.tolist()
        if 'Released_Year' in X.columns:
            self.max_year_ = X['Released_Year'].max()
        return self

    def transform(self, X):
        X_transformed = X.copy()
        if 'Released_Year' in X_transformed.columns and self.max_year_ > 0:
            X_transformed['Movie_Age'] = self.max_year_ - X_transformed['Released_Year']
        for genre in self.top_genres_:
            X_transformed[f'is_{genre}'] = X_transformed['Genre'].str.contains(genre, case=False, na=False).astype(int)
        if 'Meta_score' in X_transformed.columns and 'No_of_Votes' in X_transformed.columns:
            X_transformed['Metascore_x_Votes'] = X_transformed['Meta_score'] * X_transformed['No_of_Votes']
        X_transformed = X_transformed.drop(columns=['Genre'], errors='ignore')
        return X_transformed




### Engenharia de Features com `FeatureCreator`
Aqui criamos um transformador customizado para enriquecer os dados de filmes antes de treinar o modelo.  
O objetivo é extrair variáveis mais informativas a partir das já existentes.

- **Top Gêneros**: a coluna `Genre` pode conter múltiplos valores. Extraímos os `top_n` gêneros mais frequentes e criamos variáveis binárias (`is_Drama`, `is_Action`, etc.) indicando se o filme pertence a eles. Isso ajuda o modelo a capturar relações entre gênero e nota do IMDB.  
- **Idade do Filme**: calculamos `Movie_Age` como a diferença entre o ano mais recente do dataset e o ano de lançamento. Filmes mais antigos tendem a acumular votos e avaliações diferentes dos mais novos.  
- **Interação entre notas e popularidade**: criamos a variável `Metascore_x_Votes`, que combina a avaliação da crítica (`Meta_score`) com o número de votos do público (`No_of_Votes`). Essa interação pode ser um forte preditor da nota do IMDB, pois reflete a percepção crítica e a representatividade da amostra de votos.  
- Ao final, removemos a coluna original `Genre`, já que sua informação foi decomposta em variáveis mais úteis.

Esse tipo de **engenharia de features direcionada** geralmente aumenta a capacidade preditiva de modelos baseados em árvores, como `GradientBoostingRegressor`.


In [8]:
def fill_gross_with_revenue(df):
    df_copy = df.copy()
    gross_mean = df_copy['Gross'].mean()
    revenue_mean = df_copy['Revenue'].mean()
    factor = revenue_mean / gross_mean if pd.notna(gross_mean) and pd.notna(revenue_mean) and gross_mean > 0 else 1.0
    df_copy['Gross'] = df_copy['Gross'].fillna(df_copy['Revenue'] / factor)
    return df_copy

### Função de imputação de `Gross` com base em `Revenue`
Muitos filmes não têm valor de bilheteria (`Gross`) registrado, mas têm receita total (`Revenue`).  
A estratégia aqui é preencher os valores ausentes de `Gross` estimando-os proporcionalmente a `Revenue`.

- Calculamos um **fator de ajuste** como a razão entre as médias de $Revenue$ e $Gross.  
- Quando $Gross$ é nulo, substituímos por $\frac{Revenue}{ fator}$.  
- Essa técncia reduz perda de informação sem simplesmente descartar linhas ou usar média bruta, mantendo coerência com a escala financeira dos filmes.  

Essa abordagem evita **vazamnto de dados**, pq é aplicada dentro de cada partição do treino/validaçao no pipeline, garantindo que estatísticas de imputação sejam aprendidas apenas a partir dos dados disponiveis no conjunto de treino.


In [9]:
df_filled = fill_gross_with_revenue(df)

gross_index = df_filled.columns.get_loc('Gross')
cols_to_drop_after_gross = df_filled.columns[gross_index + 1:].tolist()
df_processed = df_filled.drop(columns=cols_to_drop_after_gross)

target = 'IMDB_Rating'
columns_to_drop = ['Series_Title', 'Overview', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', target]
X = df_processed.drop(columns=columns_to_drop, errors='ignore')
y = df_processed[target]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, make_column_selector(dtype_include=np.number)),
    ('cat', categorical_transformer, make_column_selector(dtype_include=object))
    ],
    remainder='drop'
)

final_model = Pipeline(steps=[
    ('feature_creator', FeatureCreator()),
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

cv_splitter = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(final_model, X, y, cv=cv_splitter, scoring='r2', n_jobs=-1)

print(f"Score R² (validação cruzada): {np.mean(scores):.4f}")

print("A treinar o modelo final com todos os dados...")
final_model.fit(X, y)
print("Treino concluído.")

model_filename = '../models/melhor_modelo.pkl'
joblib.dump(final_model, model_filename)

print(f"Modelo salvo com sucesso no ficheiro: {model_filename}")

Score R² (validação cruzada): 0.5264
A treinar o modelo final com todos os dados...
Treino concluído.
Modelo salvo com sucesso no ficheiro: ../models/melhor_modelo.pkl


In [10]:
import pandas as pd
import numpy as np
import joblib

model_filename = '../models/melhor_modelo.pkl'

dados_para_previsao = {
    'Series_Title': 'The Shawshank Redemption',
    'Released_Year': '1994',
    'Certificate': 'A',
    'Runtime': '142 min',
    'Genre': 'Drama',
    'Overview': '''Two imprisoned men bond over a number of years,
finding solace and eventual redemption through acts of common
decency.''',
    'Meta_score': 80.0,
    'Director': 'Frank Darabont',
    'Star1': 'Tim Robbins',
    'Star2': 'Morgan Freeman',
    'Star3': 'Bob Gunton',
    'Star4': 'William Sadler',
    'No_of_Votes': 2343110,
    'Gross': '28,341,469'
}


try:
    loaded_model = joblib.load(model_filename)
except FileNotFoundError:
    print(f"Erro: O ficheiro do modelo '{model_filename}' não foi encontrado. Por favor, execute primeiro o script de treino para salvar o modelo.")
    exit()

input_df = pd.DataFrame([dados_para_previsao])
input_df['Released_Year'] = pd.to_numeric(input_df['Released_Year'])
input_df['Runtime'] = input_df['Runtime'].str.replace(' min', '').astype(int)
input_df['Gross'] = input_df['Gross'].str.replace(',', '').astype(float)

predicted_rating = loaded_model.predict(input_df)

print(f"Filme: {dados_para_previsao['Series_Title']}")
print(f"Previsão da Nota IMDB: {predicted_rating[0]:.4f}")

Filme: The Shawshank Redemption
Previsão da Nota IMDB: 8.9847


### Treinamento e Salvamento
Após validação, treinamos o modelo em todos os dados disponíveis e o salvamos com `joblib`.  
Esse passo garante reprodutibilidade e facilita o reuso do modelo em produção ou em análises futuras.
